In [1]:
import math
import numpy as np
import scipy.sparse as sp
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

# 1. 학습 설정

In [4]:
config = {
    'data_path' : "/opt/ml/input/data/train" , # 데이터 경로
    'model_path' : "../model",


    'submission_path' : "../submission",
    'submission_name' : 'Ensembel_v1_submission.csv',

    'candidate_item_num' : 50,
    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

# 2. 데이터 전처리

In [5]:
class MakeMatrixDataSet():
    """
    MatrixDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, 'train_ratings.csv'))
        
        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('item')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('user')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['item'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['user'].apply(lambda x : self.user_encoder[x])

        self.user_train, self.user_valid = self.generate_sequence_data()

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['time']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def make_matrix(self, user_list, train = True):
        """
        user_item_dict를 바탕으로 행렬 생성
        """
        mat = torch.zeros(size = (user_list.size(0), self.num_item))
        for idx, user in enumerate(user_list):
            if train:
                mat[idx, self.user_train[user.item()]] = 1
            else:
                mat[idx, self.user_train[user.item()] + self.user_valid[user.item()]] = 1
        return mat

    def make_sparse_matrix(self, test = False):
        X = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        
        for user in self.user_train.keys():
            item_list = self.user_train[user]
            X[user, item_list] = 1.0
        
        if test:
            for user in self.user_valid.keys():
                item_list = self.user_valid[user]
                X[user, item_list] = 1.0

        return X.tocsr()

In [6]:
class AEDataSet(Dataset):
    def __init__(self, num_user):
        self.num_user = num_user
        self.users = [i for i in range(num_user)]

    def __len__(self):
        return self.num_user

    def __getitem__(self, idx): 
        user = self.users[idx]
        return torch.LongTensor([user])

# 3. 모델

In [ ]:
class EASE():
    def __init__(self, X, reg):
        self.X = self._convert_sp_mat_to_sp_tensor(X)
        self.reg = reg
    
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res
    
    def fit(self):
        '''

        진짜 정말 간단한 식으로 모델을 만듬

        '''
        G = self.X.to_dense().t() @ self.X.to_dense()
        diagIndices = torch.eye(G.shape[0]) == 1
        G[diagIndices] += self.reg

        P = G.inverse()
        B = P / (-1 * P.diag())
        B[diagIndices] = 0

        self.pred = self.X.to_dense() @ B

In [ ]:
def swish(x):
    return x.mul(torch.sigmoid(x))

def log_norm_pdf(x, mu, logvar):
    return -0.5*(logvar + np.log(2 * np.pi) + (x - mu).pow(2) / logvar.exp())

class CompositePrior(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, mixture_weights=[3/20, 3/4, 1/10]):
        super(CompositePrior, self).__init__()
        
        self.mixture_weights = mixture_weights
        
        self.mu_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.mu_prior.data.fill_(0)
        
        self.logvar_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_prior.data.fill_(0)
        
        self.logvar_uniform_prior = nn.Parameter(torch.Tensor(1, latent_dim), requires_grad=False)
        self.logvar_uniform_prior.data.fill_(10)
        
        self.encoder_old = Encoder(hidden_dim, latent_dim, input_dim)
        self.encoder_old.requires_grad_(False)
        
    def forward(self, x, z):

        post_mu, post_logvar = self.encoder_old(x, dropout_rate = 0)

        stnd_prior = log_norm_pdf(z, self.mu_prior, self.logvar_prior)
        post_prior = log_norm_pdf(z, post_mu, post_logvar)
        unif_prior = log_norm_pdf(z, self.mu_prior, self.logvar_uniform_prior)
        
        gaussians = [stnd_prior, post_prior, unif_prior]
        gaussians = [g.add(np.log(w)) for g, w in zip(gaussians, self.mixture_weights)]

        density_per_gaussian = torch.stack(gaussians, dim=-1)

        return torch.logsumexp(density_per_gaussian, dim=-1)

    
class Encoder(nn.Module):
    def __init__(self, hidden_dim, latent_dim, input_dim, eps=1e-1):
        super(Encoder, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ln2 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.ln3 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.ln4 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.ln5 = nn.LayerNorm(hidden_dim, eps=eps)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x, dropout_rate):
        norm = x.pow(2).sum(dim=-1).sqrt()
        x = x / norm[:, None]
    
        x = F.dropout(x, p=dropout_rate, training=self.training)
        
        h1 = self.ln1(swish(self.fc1(x)))
        h2 = self.ln2(swish(self.fc2(h1) + h1))
        h3 = self.ln3(swish(self.fc3(h2) + h1 + h2))
        h4 = self.ln4(swish(self.fc4(h3) + h1 + h2 + h3))
        h5 = self.ln5(swish(self.fc5(h4) + h1 + h2 + h3 + h4))
        return self.fc_mu(h5), self.fc_logvar(h5)


class RecVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim = 600, latent_dim = 200):
        super(RecVAE, self).__init__()

        self.encoder = Encoder(hidden_dim, latent_dim, input_dim)
        self.prior = CompositePrior(hidden_dim, latent_dim, input_dim)
        self.decoder = nn.Linear(latent_dim, input_dim)
        
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, user_ratings, beta=None, gamma=0.0005, dropout_rate=0.7, calculate_loss=True):
        mu, logvar = self.encoder(user_ratings, dropout_rate=dropout_rate)    
        z = self.reparameterize(mu, logvar)
        x_pred = self.decoder(z)

        if calculate_loss:
            if gamma:
                norm = user_ratings.sum(dim=-1)
                kl_weight = gamma * norm
            elif beta:
                kl_weight = beta

            mll = (F.log_softmax(x_pred, dim=-1) * user_ratings).sum(dim=-1).mean()
            kld = (log_norm_pdf(z, mu, logvar) - self.prior(user_ratings, z)).sum(dim=-1).mul(kl_weight).mean()
            negative_elbo = -(mll - kld)
            
            return (mll, kld), negative_elbo
            
        else:
            return x_pred

    def update_prior(self):
        self.prior.encoder_old.load_state_dict(deepcopy(self.encoder.state_dict()))

# 4. 학습 함수

In [8]:
def get_ndcg(pred_list, true_list):
    idcg = sum((1 / np.log2(rank + 2) for rank in range(1, len(pred_list))))
    dcg = 0
    for rank, pred in enumerate(pred_list):
        if pred in true_list:
            dcg += 1 / np.log2(rank + 2)
    ndcg = dcg / idcg
    return ndcg

# hit == recall == precision
def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit

def evaluate(model1, model2, RecVAE, X, user_train, user_valid, candidate_cnt):
    RecVAE.eval()

    NDCG = 0.0 # NDCG@10
    HIT = 0.0 # HIT@10

    recon_mat1 = model1.pred.cpu()
    score1 = recon_mat1 * torch.from_numpy(1 - X)
    rec_list1 = score1.argsort(dim = 1)

    recon_mat2 = model2.pred.T.cpu()
    score2 = recon_mat2 * torch.from_numpy(1 - X)
    rec_list2 = score2.argsort(dim = 1)

    recon_mat3 = RecVAE(torch.from_numpy(X).to(device), calculate_loss = False).cpu()
    score3 = recon_mat3 * torch.from_numpy(1 - X)
    rec_list3 = score3.argsort(dim = 1)

    score_li = np.array([1 / np.log2(rank + 2) for rank in range(0, candidate_cnt)])

    for user, (rec1, rec2, rec3) in enumerate(zip(rec_list1, rec_list2, rec_list3)):
        uv = user_valid[user]

        # ranking
        rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

        items = list(set(rec1 + rec2 + rec3))

        movie_df = pd.DataFrame(index = items)
        movie_df.loc[rec1, 'rec1_score'] = score_li
        movie_df.loc[rec2, 'rec2_score'] = score_li
        movie_df.loc[rec3, 'rec3_score'] = score_li
        movie_df = movie_df.fillna(min(score_li))
        movie_df['total_score'] = movie_df['rec1_score'] + movie_df['rec2_score'] + movie_df['rec3_score']
        movie_df = movie_df.sort_values('total_score', ascending = False)
        up = movie_df.index.tolist()[:10]

        NDCG += get_ndcg(pred_list = up, true_list = uv)
        HIT += get_hit(pred_list = up, true_list = uv)

    NDCG /= len(user_train)
    HIT /= len(user_train)

    return NDCG, HIT


def predict(model1, model2, RecVAE, X, candidate_cnt):
    RecVAE.eval()

    recon_mat1 = model1.pred.cpu()
    score1 = recon_mat1 * torch.from_numpy(1 - X)
    rec_list1 = score1.argsort(dim = 1)

    recon_mat2 = model2.pred.T.cpu()
    score2 = recon_mat2 * torch.from_numpy(1 - X)
    rec_list2 = score2.argsort(dim = 1)

    recon_mat3 = RecVAE(torch.from_numpy(X).to(device), calculate_loss = False).cpu()
    score3 = recon_mat3 * torch.from_numpy(1 - X)
    rec_list3 = score3.argsort(dim = 1)

    score_li = np.array([1 / np.log2(rank + 2) for rank in range(0, candidate_cnt)])

    user2rec = {}

    for user, (rec1, rec2, rec3) in enumerate(zip(rec_list1, rec_list2, rec_list3)):
        # ranking
        rec1 = rec1[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec2 = rec2[-candidate_cnt:].cpu().numpy().tolist()[::-1]
        rec3 = rec3[-candidate_cnt:].cpu().numpy().tolist()[::-1]

        items = list(set(rec1 + rec2 + rec3))

        movie_df = pd.DataFrame(index = items)
        movie_df.loc[rec1, 'rec1_score'] = score_li
        movie_df.loc[rec2, 'rec2_score'] = score_li
        movie_df.loc[rec3, 'rec3_score'] = score_li
        movie_df = movie_df.fillna(min(score_li))
        movie_df['total_score'] = movie_df['rec1_score'] + movie_df['rec2_score'] + movie_df['rec3_score']
        movie_df = movie_df.sort_values('total_score', ascending = False)
        up = movie_df.index.tolist()[:10]

        user2rec[user] = up

    return user2rec

# 5. 학습

In [9]:
make_matrix_data_set = MakeMatrixDataSet(config = config)
user_train, user_valid = make_matrix_data_set.get_train_valid_data()
X = make_matrix_data_set.make_sparse_matrix()

In [10]:
model1 = EASE(X = X, reg = 750)
model1.fit()

In [11]:
model2 = EASE(X = X.T, reg = 4400)
model2.fit()

In [12]:
model3 = RecVAE(
    input_dim = make_matrix_data_set.num_item,).to(device)

model3.load_state_dict(torch.load(os.path.join(config.model_path, 'RecVAE_v3.pt')))

In [ ]:
for candidate_cnt in [3] + [5 * i for i in range(1, 21)]:
    ndcg, hit = evaluate(
        model1 = model1, 
        model2 = model2, 
        RecVAE = model3, 
        X = X.todense(), 
        user_train = user_train, 
        user_valid = user_valid, 
        candidate_cnt = candidate_cnt)
    print(f'candidate_cnt: {candidate_cnt}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

# 6. 예측

In [ ]:
make_matrix_data_set = MakeMatrixDataSet(config = config)
X_test = make_matrix_data_set.make_sparse_matrix(test = True)

In [ ]:
model1 = EASE(X = X_test, reg = 750)
model1.fit()

In [ ]:
model2 = EASE(X = X_test.T, reg = 4400)
model2.fit()

In [ ]:
model3 = RecVAE(
    input_dim = make_matrix_data_set.num_item,).to(device)

model3.load_state_dict(torch.load(os.path.join(config.model_path, 'RecVAE_v3.pt')))

In [ ]:
user2rec_list = predict(
    model1 = model1, 
    model2 = model2, 
    RecVAE = model3, 
    X = X_test.todense(),
    candidate_cnt = 50)

submision = []
users = [i for i in range(0, make_matrix_data_set.num_user)]
for user in users:
    rec_item_list = user2rec_list[user]
    for item in rec_item_list:
        submision.append(
            {   
                'user' : make_matrix_data_set.user_decoder[user],
                'item' : make_matrix_data_set.item_decoder[item],
            }
        )

submision = pd.DataFrame(submision)

In [ ]:
submision.to_csv(os.path.join(config.submission_path, config.submission_name), index=False)